In [ ]:
import os
import sys
import pandas as pd

import skops.io as sio
import base64

sys.path.append(os.path.join(os.getcwd(), '..'))

from schemas.model_selection import ModelSelectionRequest
from agents.model_selection_agent import run_model_agent
from schemas.feature import FeatureSpec
from sklearn.metrics import accuracy_score

In [2]:
selected_features = [FeatureSpec(name='Fare', dtype='numeric', origin='raw', transformer='StandardScaler', params={}, importance=0.25), FeatureSpec(name='SibSp', dtype='numeric', origin='raw', transformer='StandardScaler', params={}, importance=0.15), FeatureSpec(name='Pclass', dtype='numeric', origin='raw', transformer='StandardScaler', params={}, importance=0.1), FeatureSpec(name='Parch', dtype='numeric', origin='raw', transformer='StandardScaler', params={}, importance=0.05)]
preprocessing_code = '...'

In [3]:
selected_feature_names = [feature.name for feature in selected_features]

selected_feature_names

['Fare', 'SibSp', 'Pclass', 'Parch']

In [4]:
def decode_pipeline(base64_blob):
    binary_blob = base64.b64decode(base64_blob)
    pipe = sio.loads(binary_blob)
    return pipe

In [ ]:
decoded_pipeline = decode_pipeline(preprocessing_code)

str_decoded_pipeline = str(decoded_pipeline)

str_decoded_pipeline = "\n".join([line for line in str_decoded_pipeline.split("\n") if line.strip() != ""])

str_decoded_pipeline

"ColumnTransformer(transformers=[('num',\n                                 Pipeline(steps=[('impute',\n                                                  SimpleImputer(strategy='median')),\n                                                 ('scale', StandardScaler())]),\n                                 ['Fare', 'SibSp', 'Pclass', 'Parch']),\n                                ('cat',\n                                 Pipeline(steps=[('impute',\n                                                  SimpleImputer(strategy='most_frequent')),\n                                                 ('encode',\n                                                  OneHotEncoder(handle_unknown='ignore'))]),\n                                 [])])"

In [6]:
df = pd.read_csv("../datasets/titanic.csv").head(10)
data_sample = df.to_dict(orient="list")

In [ ]:
from schemas.shared import Metadata

req = ModelSelectionRequest(
    metadata=Metadata(
        dataset_name="Titanic",
        problem_type="classification",
        target_column="Survived"
    ),
    data=data_sample,
    selected_features=selected_feature_names,
)

In [8]:
resp = run_model_agent(req, preprocessing_code)

2025-05-29 12:11:48.262 | INFO     | agents.model_selection_agent:run_model_agent:98 - Processing request for dataset 'Titanic'
2025-05-29 12:11:48.263 | INFO     | agents.model_selection_agent:_build_prompt:52 - Building the prompt for model selection.
2025-05-29 12:11:48.263 | INFO     | agents.model_selection_agent:run_model_agent:102 - Prompt length: 154 characters
2025-05-29 12:11:51.864 | INFO     | agents.model_selection_agent:run_model_agent:107 - LLM response received successfully
2025-05-29 12:11:51.877 | INFO     | agents.model_selection_agent:run_model_agent:113 - Pipeline serialized to {len(pipe_blob)} bytes
2025-05-29 12:11:51.878 | INFO     | agents.model_selection_agent:run_model_agent:119 - Model selection completed successfully


In [9]:
resp[0]
resp[1]

'...'

In [10]:
resp[0]

ModelSelectionResponse(model_name=<ModelEnum.RANDOMFOREST: 'RandomForest'>, hyperparameters={}, reasoning='Random Forest is a versatile and powerful model for classification tasks. It can handle both numerical and categorical features well, making it suitable for the selected features in the Titanic dataset. Additionally, Random Forest provides good accuracy and feature importance, which can aid in interpretability.')

In [11]:
df = pd.read_csv("../datasets/titanic.csv")

pipeline = decode_pipeline(resp[1])

X = df.drop(columns=["Survived"])
y = df["Survived"]

pipeline.fit(X, y)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scale',
                                                                   StandardScaler())]),
                                                  ['Fare', 'SibSp', 'Pclass',
                                                   'Parch']),
                                                 ('cat',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encode',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  [])])),
                ('model', RandomForestClassifier())])

In [ ]:
y_pred = pipeline.predict(X)

accuracy = accuracy_score(y, y_pred)
print(f"Accuracy of the model: {accuracy:.2f}")

Accuracy of the model: 0.84
